In [6]:
from obspy import UTCDateTime
import numpy as np
import os
import glob

# 读取震相，输出npy文件

In [7]:
readpath=r'..\data\ph'
savepath=r'..\data\train_data'
files=glob.glob(r'%s\*.phase' %readpath)

In [8]:
phase = {}
for file in files:
    fn=os.path.basename(file).split('.phase')[0]  #获取文件名
    f = open(file)
    lines = f.readlines()
    for i in range(3):
        lines.pop(0)
    for line in lines:
        if line.__eq__('#Station Magnitudes:\n'):
            break
        words = line.split()
        arrival = UTCDateTime(words[4]+' '+words[5])
        st = words[0].split('.')[1]
        #words[7] is weight. We skip the loop when the weight is zero and distance larger than 2°.
        if (words[7]!='1.0') or (float(words[1])>2):
            continue
        if (words[3].rfind('Pn')!=-1):
            phasetype = 'P'
        elif (words[3].rfind('Pg')!=-1):
            phasetype = 'P'
        elif (words[3].rfind('Sg')!=-1):
            phasetype = 'S'
        else:
            continue
        #Initializing.
        if (not phase.__contains__(fn)):
            phase[fn] = {}
        if (not phase[fn].__contains__(st)):
            phase[fn][st] = {}
        if (phase[fn][st].__contains__(phasetype)):
            phase[fn][st][phasetype] = min(arrival,phase[fn][st][phasetype])
        else:
            phase[fn][st][phasetype] = arrival
    #break
np.save(savepath+'\\'+'seismic_phases',phase)